<a href="https://colab.research.google.com/github/aml7hawaiiedu/CCAPLandCoverProject/blob/main/CCAP_UNET_processing_Fall2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import shutil
import hashlib

In [3]:
# # OR Unzipping to a specific directory
!mkdir /content/CCAP

In [27]:
!mkdir '/content/CCAP/Hawaii_2005_002'
!mkdir '/content/CCAP/Hawaii_2005_002/Images'
!mkdir '/content/CCAP/Hawaii_2005_002/Labels'

In [ ]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/Hawaii_Island_Update/Export/Hawaii_2005_002.zip -d /content/CCAP/HawaiI_2005_002

Next two blocks of code are probably unnecessary but run them for a sanity check.

In [9]:
# original images and labels folder: how many files are there?

# Count how many files are in the images folder
org_img = '/content/CCAP/content/Hawaii_2005_002/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')

# Count how many files are in the labels folder
org_label = '/content/CCAP/content/Hawaii_2005_002/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 1476
The number of files in the folder is: 1476


In [10]:
# # original images and labels folder: are any of them duplicated?

# # Check the images folder for duplicates
# org_img = '/content/CCAP/content/Hawaii_2005_002/Images'
# file_hashes = {}
# for org_img_filename in os.listdir(org_img):
#     file_path = os.path.join(org_img, org_img_filename)
#     if os.path.isfile(file_path):
#         with open(file_path, 'rb') as file:
#             file_content = file.read()
#             file_hash = hashlib.md5(file_content).hexdigest()
#         if file_hash in file_hashes:
#             print(f"Duplicate found: {org_img_filename} is a duplicate of {file_hashes[file_hash]}")
#         else:
#             file_hashes[file_hash] = org_img_filename
# print("Duplicate check completed.")

# # Check the label folder for duplicates
# org_label = '/content/CCAP/content/Hawaii_2005_002/Labels'
# file_hashes = {}
# for org_label_filename in os.listdir(org_label):
#     file_path = os.path.join(org_label, org_label_filename)
#     if os.path.isfile(file_path):
#         with open(file_path, 'rb') as file:
#             file_content = file.read()
#             file_hash = hashlib.md5(file_content).hexdigest()
#         if file_hash in file_hashes:
#             print(f"Duplicate found: {org_label_filename} is a duplicate of {file_hashes[file_hash]}")
#         else:
#             file_hashes[file_hash] = org_label_filename
# print("Duplicate check completed.")

Duplicate check completed.
Duplicate check completed.


In [12]:
df = pd.read_csv('/content/CCAP/content/Hawaii_2005_002/Hawaii_2005_002.csv')

In [ ]:
df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent
0,513,513,1,1,1,673774.792178,675003.549951,2.349134e+06,2.350363e+06,0,1.0
1,1025,513,513,1,2,675003.581956,676232.397422,2.349134e+06,2.350363e+06,0,1.0
2,1537,513,1025,1,3,676232.402366,677461.226108,2.349134e+06,2.350363e+06,0,1.0
3,2049,513,1537,1,4,677461.191368,678689.981511,2.349134e+06,2.350363e+06,0,1.0
4,2561,513,2049,1,5,678690.051385,679918.820215,2.349134e+06,2.350363e+06,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
3857,25601,9851,25089,9339,1076,733985.987134,735214.794851,2.326723e+06,2.327952e+06,0,1.0
3858,26113,9851,25601,9339,1077,735214.804523,736443.648874,2.326723e+06,2.327952e+06,0,1.0
3859,26625,9851,26113,9339,1078,736443.564339,737672.387927,2.326723e+06,2.327952e+06,0,1.0
3860,27137,9851,26625,9339,1079,737672.401853,738901.225655,2.326723e+06,2.327952e+06,0,1.0


In [13]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [14]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent
52,10753,513,10241,1,21,203422.783910,204651.567817,2.242729e+06,2.243958e+06,15,0.003197
120,9729,1025,9217,513,55,200965.101972,202193.917191,2.241500e+06,2.242729e+06,15,0.010220
127,10241,1025,9729,513,56,202193.969053,203422.747766,2.241501e+06,2.242729e+06,15,0.007744
229,10241,1537,9729,1025,92,202193.964501,203422.776126,2.240272e+06,2.241500e+06,13,0.001568
230,10241,1537,9729,1025,92,202193.964501,203422.776126,2.240272e+06,2.241500e+06,15,0.021610
...,...,...,...,...,...,...,...,...,...,...,...
2563,18302,10753,17790,10241,756,221540.306850,222769.117460,2.218153e+06,2.219382e+06,15,0.001030
3428,8705,15361,8193,14849,1061,198507.490908,199736.304291,2.207094e+06,2.208323e+06,16,0.013378
3429,8705,15361,8193,14849,1061,198507.490908,199736.304291,2.207094e+06,2.208323e+06,18,0.005234
3437,9217,15361,8705,14849,1062,199736.312984,200965.117609,2.207094e+06,2.208323e+06,16,0.004280


In [15]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [16]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name
52,10753,513,10241,1,21,203422.783910,204651.567817,2.242729e+06,2.243958e+06,15,0.003197,10241_1.tif
120,9729,1025,9217,513,55,200965.101972,202193.917191,2.241500e+06,2.242729e+06,15,0.010220,9217_513.tif
127,10241,1025,9729,513,56,202193.969053,203422.747766,2.241501e+06,2.242729e+06,15,0.007744,9729_513.tif
229,10241,1537,9729,1025,92,202193.964501,203422.776126,2.240272e+06,2.241500e+06,13,0.001568,9729_1025.tif
230,10241,1537,9729,1025,92,202193.964501,203422.776126,2.240272e+06,2.241500e+06,15,0.021610,9729_1025.tif
...,...,...,...,...,...,...,...,...,...,...,...,...
2563,18302,10753,17790,10241,756,221540.306850,222769.117460,2.218153e+06,2.219382e+06,15,0.001030,17790_10241.tif
3428,8705,15361,8193,14849,1061,198507.490908,199736.304291,2.207094e+06,2.208323e+06,16,0.013378,8193_14849.tif
3429,8705,15361,8193,14849,1061,198507.490908,199736.304291,2.207094e+06,2.208323e+06,18,0.005234,8193_14849.tif
3437,9217,15361,8705,14849,1062,199736.312984,200965.117609,2.207094e+06,2.208323e+06,16,0.004280,8705_14849.tif


In [ ]:

# Copy the useable images files
# Define the source and destination directories
source_images = '/content/CCAP/content/Hawaii_2005_002/Images'
destination_images = '/content/CCAP/Hawaii_2005_002/Images'

# Iterate through the 'tif_name' column in the subset_df
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)

    try:
        # Copy the file from the source to the destination folder
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

In [ ]:
# Copy the useable labels files
# Define the source and destination directories
source_labels = '/content/CCAP/content/Hawaii_2005_002/Labels'
destination_labels = '/content/CCAP/Hawaii_2005_002/Labels'

# Iterate through the 'tif_name' column in the subset_df
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)

    try:
        # Copy the file from the source to the destination folder
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

In [21]:
# NEW images and labels folder: how many files are there?

# Count how many files are in the new images folder
new_img = '/content/CCAP/Hawaii_2005_002/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')

# Count how many files are in the new labels folder
new_label = '/content/CCAP/Hawaii_2005_002/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 116
The number of files in the folder is: 116


In [22]:
# # new images and labels folder: are any of them duplicated?

# # Check the new images folder for duplicates
# new_img = '/content/CCAP/HawaiI_2005_001/Images'
# file_hashes = {}
# for new_img_filename in os.listdir(new_img):
#     file_path = os.path.join(new_img, new_img_filename)
#     if os.path.isfile(file_path):
#         with open(file_path, 'rb') as file:
#             file_content = file.read()
#             file_hash = hashlib.md5(file_content).hexdigest()
#         if file_hash in file_hashes:
#             print(f"Duplicate found: {new_img_filename} is a duplicate of {file_hashes[file_hash]}")
#         else:
#             file_hashes[file_hash] = new_img_filename
# print("Duplicate check completed.")

# # Check the new label folder for duplicates
# new_label = '/content/CCAP/HawaiI_2005_001/Labels'
# file_hashes = {}
# for new_label_filename in os.listdir(new_label):
#     file_path = os.path.join(new_label, new_label_filename)
#     if os.path.isfile(file_path):
#         with open(file_path, 'rb') as file:
#             file_content = file.read()
#             file_hash = hashlib.md5(file_content).hexdigest()
#         if file_hash in file_hashes:
#             print(f"Duplicate found: {new_label_filename} is a duplicate of {file_hashes[file_hash]}")
#         else:
#             file_hashes[file_hash] = new_label_filename
# print("Duplicate check completed.")

Duplicate check completed.
Duplicate check completed.


In [23]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/Hawaii_2005_002_subset.csv'

# Export the DataFrame to a CSV file
subset_df.to_csv(csv_file_path, index=False)

print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/HawaiI_2005_001_subset.csv


In [24]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/CCAP/Hawaii_2005_002'

# Define the path for the ZIP file
zip_file_path = '/content/Hawaii_2005_002_subset.zip'

# Create a ZIP archive of the folder
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)

print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/HawaiI_2005_001_subset.zip


In [25]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata'

# Move the ZIP file to Google Drive
shutil.move(zip_file_path, drive_folder)

print(f'ZIP file moved to Google Drive: {drive_folder}')

# Move the CSV file to Google Drive
shutil.move(csv_file_path, drive_folder)

print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata
CSV file moved to Google Drive: /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata


In [ ]:
!mkdir '/content/CCAP/Hawaii_2005_003'
!mkdir '/content/CCAP/Hawaii_2005_003/Images'
!mkdir '/content/CCAP/Hawaii_2005_003/Labels'

In [ ]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/Hawaii_Island_Update/Export/Hawaii_2005_003.zip -d /content/CCAP/HawaiI_2005_003